# Single asset strategy test



In [1]:

import sys, os
from pathlib import Path


PROJECT_ROOT = Path("..").resolve()
SRC_DIR = PROJECT_ROOT / "src"
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))


DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)


from hedge.data import (
    DataScheme,
    load_ohlcv_by_code,
    download_binance_ohlcv_csv,
    ensure_ohlcv_dataset,   
)
from hedge.run import train_on_csv, run_on_csv
from hedge.strategies import SingleAssetMACrossover
from hedge.portfolio import CASH_COL
from hedge import viz


import pandas as pd
import numpy as np


In [13]:
# --- USER PARAMETERS ---
SYMBOL      = "BTCUSDT"     
INTERVAL    = "1h"         
START       = "2024-01-01"  
END         = None          
TRAIN_END   = "2024-06-01"  


STRAT_NAME = "single-asset-ma-crossover"
STRAT_PARAMS = dict(
    asset=SYMBOL,
    price_field="close",
    fast_window=20,
    slow_window=50,
    w_long=1.0,
    w_short=0.0,      
    clip_per_asset=1.0,
    warmup_bars=50,   
    trade_lag_bars=1,
    rebalance_every=None,
)

# raw CSV
CSV_PATH = DATA_DIR / f"{SYMBOL}_{INTERVAL}.csv"

FORCE_DOWNLOAD = False

In [14]:

code, path = ensure_ohlcv_dataset(SYMBOL, INTERVAL, START, END, data_dir="data")

print(code, path)
# Load & validate
df = load_ohlcv_by_code(code, data_dir="data")
print(df.head(3))
print(df.tail(3))
print("Rows:", len(df), "| Index tz:", df.index.tz)

if viz.plot_price_or_candles:
    viz.plot_price_or_candles(df, asset=SYMBOL, mode="candles")  # or kind="line"
else:
    ax = df["close"].plot(title=f"{SYMBOL} {INTERVAL} close")
    ax.set_xlabel("time"); ax.set_ylabel("price")


RuntimeError: Binance returned rows, but all were invalid/NaN.